In [1]:
!wget https://raw.githubusercontent.com/babu-thomas/movielens-posters/master/movie_url.csv
!wget https://raw.githubusercontent.com/babu-thomas/movielens-posters/master/movie_poster.csv

--2021-10-12 10:29:20--  https://raw.githubusercontent.com/babu-thomas/movielens-posters/master/movie_url.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94025 (92K) [text/plain]
Saving to: ‘movie_url.csv.1’

movie_url.csv.1     100%[===================>]  91.82K  --.-KB/s    in 0.01s   

2021-10-12 10:29:20 (9.22 MB/s) - ‘movie_url.csv.1’ saved [94025/94025]

--2021-10-12 10:29:20--  https://raw.githubusercontent.com/babu-thomas/movielens-posters/master/movie_poster.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202458 (

In [2]:
import pandas as pd
import numpy as np

In [3]:
poster_df = pd.read_csv('movie_poster.csv', header=None)

In [4]:
poster_df.set_index(0, inplace=True)
poster_df.head()

,1
0,
1,https://images-na.ssl-images-amazon.com/images...
2,https://images-na.ssl-images-amazon.com/images...
3,https://images-na.ssl-images-amazon.com/images...
4,https://images-na.ssl-images-amazon.com/images...
5,https://images-na.ssl-images-amazon.com/images...


In [14]:
url_df = pd.read_csv('movie_url.csv', header=None)
url_df.set_index(0, inplace=True)
url_df

,1
0,
1,http://www.imdb.com/title/tt0114709/?ref_=fn_a...
2,http://www.imdb.com/title/tt0113189/?ref_=fn_a...
3,http://www.imdb.com/title/tt0113101/?ref_=fn_a...
4,http://www.imdb.com/title/tt0113161/?ref_=fn_a...
5,http://www.imdb.com/title/tt0112722/?ref_=fn_a...
...,...
1678,http://www.imdb.com/title/tt0119711/?ref_=fn_a...
1679,http://www.imdb.com/title/tt0120594/?ref_=fn_a...
1680,http://www.imdb.com/title/tt0120148/?ref_=fn_a...


In [5]:
import tensorflow_datasets as tfds

movie_ds = tfds.load("movielens/100k-movies", split="train")

In [20]:
movie_arr = np.vstack(tfds.as_numpy(movie_ds))

def get_poster_url(features):
    features = features[0]
    movie_id = int(features['movie_id'])
    poster_url = None
    movie_url = None
    
    if movie_id in poster_df.index:
        poster_url = poster_df.loc[movie_id][1]
    if movie_id in url_df.index:
        movie_url= url_df.loc[movie_id][1]
    return {**features,
            'poster_url': poster_url,
            'movie_url': movie_url}

movie_arr = np.array(list(map(get_poster_url, movie_arr)))
movie_arr = [arr for arr in movie_arr if arr != None]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [24]:
movie_df = pd.DataFrame(movie_arr)
movie_df.dropna(axis=0, inplace=True)

In [25]:
movie_df['movie_id'] = movie_df['movie_id'].apply(lambda x: x.decode('utf-8'))
movie_df['movie_title'] = movie_df['movie_title'].apply(lambda x: x.decode('utf-8'))

In [29]:
movie_df

,movie_genres,movie_id,movie_title,poster_url,movie_url
0,[4],1681,You So Crazy (1994),https://images-na.ssl-images-amazon.com/images...,http://www.imdb.com/title/tt0111804/?ref_=fn_a...
1,"[4, 7]",1457,Love Is All There Is (1996),https://images-na.ssl-images-amazon.com/images...,http://www.imdb.com/title/tt0116928/?ref_=fn_a...
2,"[1, 3]",500,Fly Away Home (1996),https://images-na.ssl-images-amazon.com/images...,http://www.imdb.com/title/tt0116329/?ref_=fn_a...
3,[0],838,In the Line of Duty 2 (1987),https://images-na.ssl-images-amazon.com/images...,http://www.imdb.com/title/tt0091229/?ref_=fn_a...
4,[7],1648,"Niagara, Niagara (1997)",https://images-na.ssl-images-amazon.com/images...,http://www.imdb.com/title/tt0119780/?ref_=fn_a...
...,...,...,...,...,...
1676,[10],1573,Spirits of the Dead (Tre passi nel delirio) (1...,https://images-na.ssl-images-amazon.com/images...,http://www.imdb.com/title/tt0063715/?ref_=fn_a...
1677,[4],1611,Intimate Relations (1996),https://images-na.ssl-images-amazon.com/images...,http://www.imdb.com/title/tt0116643/?ref_=fn_a...
1679,[4],824,"Great White Hype, The (1996)",https://images-na.ssl-images-amazon.com/images...,http://www.imdb.com/title/tt6154544/?ref_=fn_a...
1680,[7],1354,Venice/Venice (1992),https://images-na.ssl-images-amazon.com/images...,http://www.imdb.com/title/tt0105729/?ref_=fn_a...


In [30]:
movie_df.isnull().sum()

movie_genres    0
movie_id        0
movie_title     0
poster_url      0
movie_url       0
dtype: int64

In [31]:
movie_df.to_csv('movielens_poster.csv', index=False)